In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import FunctionTransformer

# 1.a


### Loading Datasets


In [2]:
attendance = pd.read_csv("data/attendance.csv")
referees = pd.read_csv("data/referees.csv")
matches = pd.read_csv("data/fjelstul/matches.csv")
stadiums = pd.read_csv("data/fjelstul/stadiums.csv")
teams = pd.read_csv("data/fjelstul/teams.csv")


## Exploring Each Dataframe


In [3]:
# print("attendance df shape: ", attendance.shape)
# print("attendance df columns: ", attendance.columns)
# print("Nan count:")
# attendance.isnull().sum()


In [4]:
# print("referees df shape: ", referees.shape)
# print("referees df columns: ", referees.columns)
# print("Nan count:")
# referees.isnull().sum()


In [5]:
# print("matches df shape: ", matches.shape)
# print("matches df columns: ", matches.columns)
# print("Nan count:")
# matches.isnull().sum()


In [6]:
# print("stadiums df shape: ", stadiums.shape)
# print("stadiums df columns: ", stadiums.columns)
# print("Nan count:")
# stadiums.isnull().sum()


In [7]:
# print("teams df shape: ", teams.shape)
# print("teams df columns: ", teams.columns)
# print("Nan count:")
# teams.isnull().sum()


### Adding Stadium Capacity To Matches Dataframe based on `stadium_id`


In [8]:
matches_std = pd.merge(
    matches, stadiums[["stadium_id", "stadium_capacity"]], on="stadium_id")


### Setting MultiIndex For Attendance and Referees as Shared Index


In [9]:
attendance = attendance.set_index(["home_team", "away_team", "Date"])
referees = referees.set_index(["home_team", "away_team", "Date"])


### Checking if all indeces are mutual


In [10]:
shared_indices = (set(attendance.index.to_list())
                  & set(referees.index.to_list()))
len(shared_indices) - attendance.shape[0]


0

### Concatinating Attendance And Referees Dataframes


In [11]:
attendance_referees = pd.concat([attendance, referees], axis=1)
attendance_referees = attendance_referees.reset_index()
attendance_referees.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   home_team   964 non-null    object
 1   away_team   964 non-null    object
 2   Date        964 non-null    object
 3   Attendance  964 non-null    int64 
 4   Officials   709 non-null    object
dtypes: int64(1), object(4)
memory usage: 37.8+ KB


### Cheking if all team names in attendace dataframe are correct


In [12]:
atnd_team_df = pd.merge(
    teams[["team_name", "team_code"]],
    attendance_referees,
    left_on=["team_name"],
    right_on=["home_team"],
    how='right'
)


In [13]:
print("Teams that are not exist in teams tabel:")
atnd_team_df.loc[atnd_team_df["team_code"].isna(), "home_team"].unique()


Teams that are not exist in teams tabel:


array(['Korea Republic', 'IR Iran', "Côte d'Ivoire", 'Korea DPR',
       'Türkiye', 'China PR', 'FR Yugoslavia', 'Germany DR'], dtype=object)

### Normalizing all team names


In [14]:
team_name_map = {
    "Türkiye": "Turkey",
    "Korea Republic": "South Korea",
    "IR Iran": "Iran",
    "Côte d'Ivoire": "Ivory Coast",
    "Korea DPR": "North Korea",
    "China PR": "China",
    "FR Yugoslavia": "Yugoslavia",
    "Germany DR": "East Germany"
}
def mapper(team): return team_name_map[team] if team in team_name_map else team


In [15]:
attendance_referees["home_team"] = attendance_referees["home_team"].map(mapper)
attendance_referees["away_team"] = attendance_referees["away_team"].map(mapper)


### Seting a unique code name `sorted(home_team + away_team)` for each row because home_team and away_team are commutative


In [16]:
attendance_referees["teams_code"] = ["".join(sorted(
    home+away)) for home, away in attendance_referees[["home_team", "away_team"]].values.tolist()]
matches_std["teams_code"] = ["".join(sorted(
    home+away)) for home, away in matches_std[["home_team_name", "away_team_name"]].values.tolist()]


### Mergne Matches and Attendance


In [17]:
merged_df = pd.merge(
    matches_std,
    attendance_referees,
    left_on=["teams_code", "match_date"],
    right_on=["teams_code", "Date"],
    how='outer'
)


### Cheking If all rows are merged properly


In [18]:
mask = merged_df[["Attendance", "match_id"]].isna().any(axis=1)
merged_df[["home_team", "away_team", "home_team_name",
           "away_team_name", "Date", "match_date", "teams_code"]][mask]


,home_team,away_team,home_team_name,away_team_name,Date,match_date,teams_code
118,NaN,NaN,West Germany,Yugoslavia,NaN,1954-06-27,GWYaaaeegilmnorsstuvy
127,NaN,NaN,West Germany,Turkey,NaN,1954-06-23,GTWaeeekmnrrstuyy
130,NaN,NaN,West Germany,Turkey,NaN,1954-06-17,GTWaeeekmnrrstuyy
133,NaN,NaN,West Germany,Hungary,NaN,1954-07-04,GHWaaeegmnnrrstuyy
136,NaN,NaN,Hungary,West Germany,NaN,1954-06-20,GHWaaeegmnnrrstuyy
139,NaN,NaN,West Germany,Austria,NaN,1954-06-30,AGWaaeeimnrrssttuy
964,Germany,Hungary,NaN,NaN,1954-07-04,NaN,GHaaegmnnrruyy
965,Germany,Austria,NaN,NaN,1954-06-30,NaN,AGaaeimnrrstuy
966,Germany,Yugoslavia,NaN,NaN,1954-06-27,NaN,GYaaaegilmnorsuvy
967,Germany,Turkey,NaN,NaN,1954-06-23,NaN,GTaeekmnrruyy


### It seams that there are some mistakes in Germany team name in the attendance data, Let's correct them by repalcing Germany by West Germany because Germany team did not exist at that time `1954`


In [19]:
mask = merged_df[["match_id"]].isna().any(axis=1)
wrong_names = merged_df[["home_team", "away_team", "Date", "teams_code"]][mask]
wrong_names


,home_team,away_team,Date,teams_code
964,Germany,Hungary,1954-07-04,GHaaegmnnrruyy
965,Germany,Austria,1954-06-30,AGaaeimnrrstuy
966,Germany,Yugoslavia,1954-06-27,GYaaaegilmnorsuvy
967,Germany,Turkey,1954-06-23,GTaeekmnrruyy
968,Hungary,Germany,1954-06-20,GHaaegmnnrruyy
969,Germany,Turkey,1954-06-17,GTaeekmnrruyy


Replace the wrong name with the correct one


In [20]:
mask = attendance_referees["Date"].isin(
    wrong_names["Date"]) & attendance_referees["teams_code"].isin(wrong_names["teams_code"])

attendance_referees.loc[mask, ["home_team", "away_team"]] = attendance_referees.loc[mask, [
    "home_team", "away_team"]].replace("Germany", "West Germany")
# reshape teams_code 
attendance_referees["teams_code"] = ["".join(sorted(
    a+b)) for a, b in attendance_referees[["home_team", "away_team"]].values.tolist()]


### Let's Merge again


In [21]:
final_df = pd.merge(
    matches_std,
    attendance_referees,
    left_on=["teams_code", "match_date"],
    right_on=["teams_code", "Date"],
    how='outer'
)


In [22]:
mask = final_df[["Attendance", "match_id"]].isna().any(axis=1)
final_df[["home_team", "away_team", "home_team_name",
          "away_team_name", "Date", "match_date", "teams_code"]][mask]


,home_team,away_team,home_team_name,away_team_name,Date,match_date,teams_code


In [23]:
final_df = final_df.drop(
    ["home_team", "away_team", "teams_code", "Date"], axis=1)
final_df = final_df.rename(
    columns={"attendance": "attendance", "Officials": "officials"})
final_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 964 entries, 0 to 963
Data columns (total 40 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   key_id                     964 non-null    int64 
 1   tournament_id              964 non-null    object
 2   tournament_name            964 non-null    object
 3   match_id                   964 non-null    object
 4   match_name                 964 non-null    object
 5   stage_name                 964 non-null    object
 6   group_name                 964 non-null    object
 7   group_stage                964 non-null    int64 
 8   knockout_stage             964 non-null    int64 
 9   replayed                   964 non-null    int64 
 10  replay                     964 non-null    int64 
 11  match_date                 964 non-null    object
 12  match_time                 964 non-null    object
 13  stadium_id                 964 non-null    object
 14  stadium_na

In [24]:
final_df.to_csv("data/matches_combined_data.csv", index=False)


# Done.


# 1.b


In [25]:
players = pd.read_csv('data/fjelstul/players.csv')
squads = pd.read_csv('data/fjelstul/squads.csv')


In [26]:
players.head()


,key_id,player_id,family_name,given_name,birth_date,goal_keeper,defender,midfielder,forward,count_tournaments,list_tournaments,player_wikipedia_link
0,1,P-08891,A'Court,Alan,1934-09-30,0,0,0,1,1,1958,https://en.wikipedia.org/wiki/Alan_A%27Court
1,2,P-08589,Aaronson,Brenden,2000-10-22,0,0,0,1,1,2022,https://en.wikipedia.org/wiki/Brenden_Aaronson
2,3,P-04897,Abadzhiev,Stefan,1934-07-03,0,0,0,1,1,1966,https://en.wikipedia.org/wiki/Stefan_Abadzhiev
3,4,P-05556,Abalo,Jean-Paul,1975-06-26,0,1,0,0,1,2006,https://en.wikipedia.org/wiki/Jean-Paul_Abalo
4,5,P-08163,Abanda,Patrice,1978-08-03,0,1,0,0,1,1998,https://en.wikipedia.org/wiki/Patrice_Abanda


In [27]:
players.shape


(8485, 12)

In [28]:
squads.head()


,key_id,tournament_id,tournament_name,team_id,team_name,team_code,player_id,family_name,given_name,shirt_number,position_name,position_code
0,1,WC-1930,1930 FIFA World Cup,T-03,Argentina,ARG,P-06987,Bossio,Ángel,0,goal keeper,GK
1,2,WC-1930,1930 FIFA World Cup,T-03,Argentina,ARG,P-00287,Botasso,Juan,0,goal keeper,GK
2,3,WC-1930,1930 FIFA World Cup,T-03,Argentina,ARG,P-01435,Cherro,Roberto,0,forward,FW
3,4,WC-1930,1930 FIFA World Cup,T-03,Argentina,ARG,P-01321,Chividini,Alberto,0,defender,DF
4,5,WC-1930,1930 FIFA World Cup,T-03,Argentina,ARG,P-08552,Della Torre,José,0,defender,DF


In [29]:
squads.shape


(10973, 12)

In [30]:
players.isnull().sum()


key_id                   0
player_id                0
family_name              0
given_name               0
birth_date               1
goal_keeper              0
defender                 0
midfielder               0
forward                  0
count_tournaments        0
list_tournaments         0
player_wikipedia_link    0
dtype: int64

In [31]:
squads.isnull().sum()


key_id             0
tournament_id      0
tournament_name    0
team_id            0
team_name          0
team_code          0
player_id          0
family_name        0
given_name         0
shirt_number       0
position_name      0
position_code      0
dtype: int64

In [32]:
len(squads.player_id.unique())


8485

the same number of players in both table , so we can get the info according to any table


-----------

get columns from player dataframe


In [33]:
players_data = players[['player_id', 'given_name',
                        'family_name', 'count_tournaments', 'list_tournaments']]

get columns from squads dataframe


In [34]:
teams_names_codes = squads[['player_id', 'team_name', 'team_code']]


number of teams for each player


In [35]:
team_count = teams_names_codes.groupby(
    'player_id').agg({'team_code': pd.Series.nunique})
team_count.rename(columns={'team_code': 'team_count'}, inplace=True)


In [36]:
teams_names_codes_count = teams_names_codes.merge(team_count, on='player_id')
teams_names_codes_count


,player_id,team_name,team_code,team_count
0,P-06987,Argentina,ARG,1
1,P-00287,Argentina,ARG,1
2,P-01435,Argentina,ARG,1
3,P-01321,Argentina,ARG,1
4,P-08552,Argentina,ARG,1
...,...,...,...,...
10968,P-08129,Wales,WAL,1
10969,P-05661,Wales,WAL,1
10970,P-03434,Wales,WAL,1
10971,P-08781,Wales,WAL,1


In [37]:
plrs = teams_names_codes_count[teams_names_codes_count['team_count']
                        > 1]['player_id'].nunique()
print(f"we have {plrs} player have played in more than one team")

we have 20 player have played in more than one team


In [38]:
player_teams = players_data.merge(teams_names_codes_count, on='player_id')
player_teams


,player_id,given_name,family_name,count_tournaments,list_tournaments,team_name,team_code,team_count
0,P-08891,Alan,A'Court,1,1958,England,ENG,1
1,P-08589,Brenden,Aaronson,1,2022,United States,USA,1
2,P-04897,Stefan,Abadzhiev,1,1966,Bulgaria,BGR,1
3,P-05556,Jean-Paul,Abalo,1,2006,Togo,TGO,1
4,P-08163,Patrice,Abanda,1,1998,Cameroon,CMR,1
...,...,...,...,...,...,...,...,...
10968,P-06718,Maciej,Żurawski,2,"2002, 2006",Poland,POL,1
10969,P-03465,Szymon,Żurkowski,1,2022,Poland,POL,1
10970,P-08318,Graham,Zusi,1,2014,United States,USA,1
10971,P-05921,Andrei,Zygmantovich,1,1990,Soviet Union,SUN,1


In [39]:
def unique_list(items): return ','.join(list(dict.fromkeys(items)))

player_teams_agg = player_teams.groupby('player_id').agg({
    'given_name': "first",
    'family_name': "first",
    'count_tournaments': 'first',
    "team_count": "first",
    'list_tournaments': unique_list,
    'team_name': unique_list,
    'team_code': unique_list,
})
player_teams_agg = player_teams_agg.reset_index()
player_teams_agg


,player_id,given_name,family_name,count_tournaments,team_count,list_tournaments,team_name,team_code
0,P-00001,Harald,Schumacher,2,1,"1982, 1986",West Germany,DEU
1,P-00002,Ismail,Mohammed Sharif,1,1,1986,Iraq,IRQ
2,P-00003,not applicable,Otávio,1,1,2022,Portugal,PRT
3,P-00004,Julio César,Cortés,3,1,"1962, 1966, 1970",Uruguay,URY
4,P-00005,Guy,Vandersmissen,1,1,1982,Belgium,BEL
...,...,...,...,...,...,...,...,...
8480,P-09994,Landry,N'Guémo,2,1,"2010, 2014",Cameroon,CMR
8481,P-09996,Ramiro,Castillo,1,1,1994,Bolivia,BOL
8482,P-09997,Jonás,Gutiérrez,1,1,2010,Argentina,ARG
8483,P-09998,not applicable,Rivaldo,2,1,"1998, 2002",Brazil,BRA


In [40]:
player_teams.to_csv('data/player_teams.csv',index=False)
player_teams_agg.to_csv('data/player_teams_agg.csv',index=False)

-----------------

# 2

In [41]:
matches = pd.read_csv("data/matches_combined_data.csv")
teams = pd.read_csv("data/fjelstul/teams.csv")
players_teams_agg = pd.read_csv('data/player_teams_agg.csv')
goals = pd.read_csv('data/fjelstul/goals.csv')

### total_goals_in_match

In [42]:
matches["total_goals_in_match"] = (
    matches["home_team_score"] +
    matches["away_team_score"] +
    matches["home_team_score_penalties"] +
    matches["away_team_score_penalties"]
)

### used_capacity_ratio

In [43]:
matches["used_capacity_ratio"] = matches["Attendance"] / matches["stadium_capacity"]

### attendance_category 

In [44]:
transformer = KBinsDiscretizer(n_bins=5, encode='ordinal')
discrete_attendance = transformer.fit_transform(matches['Attendance'].values.reshape(-1, 1))
matches["attendance_category"] = discrete_attendance

### relative_attendance_category

In [45]:
bins = [0 , 0.99, 1.01, np.inf]
labels = ['underload', 'full', 'overload']
transformer = FunctionTransformer(pd.cut, kw_args={'bins': bins, 
                                                   'labels': labels, 
                                                   'retbins': False})
matches["relative_attendance_category"] = transformer.fit_transform(matches['used_capacity_ratio'])

### host_country_code

In [46]:
matches = pd.merge(
    matches,
    teams[["team_name", "team_code"]],
    left_on=["country_name"],
    right_on=["team_name"],
    how="left"
)

In [47]:
matches = matches.drop("team_name", axis=1)
matches = matches.rename(columns={"team_code": "host_country_code"})

###  match_for_host

In [48]:
condition1 = matches["host_country_code"] == matches["home_team_code"]
condition2 = matches["host_country_code"] == matches["away_team_code"]
matches["match_for_host"] = condition1 | condition2

###  tournament_year

In [49]:
matches["tournament_year"] = matches.tournament_id.str.split("-").str.get(1)

###  full_name

In [50]:
players_teams_agg['full_name'] = players_teams_agg.apply(
    lambda row: row['given_name'] + ' ' + row['family_name'] if row['given_name'] != 'not applicable' else row['family_name'],
    axis=1
)

###  short_stage_name

In [51]:
matches["short_stage_name"] = np.where(matches['group_stage'] == 1, 'group', 'knockout')

### winner_code 

In [52]:
conditions = [matches['home_team_win'].astype(bool), matches['away_team_win'].astype(bool)]
choices = [matches['home_team_code'], matches['away_team_code']]
matches['winner_code'] = np.select(conditions, choices, default=None)

### Late_goal

In [53]:
def get_max_minutes(label):
    if label == 'first half':
        return 45
    elif label == 'second half':
        return 90
    elif label == 'extra time, first half':
        return 105
    elif label == 'extra time, second half':
        return 120
    elif label == 'second half, stoppage time':
        return 90 + 13
    elif label == 'first half, stoppage time':
        return 45 + 13
    elif label == 'extra time, second half, stoppage time':
        return 120 + 13
    elif label == 'extra time, first half, stoppage time':
        return 105 + 13
    else:
        return None

def is_end_of(name, minute):
    max_minute = get_max_minutes(name)
    return max_minute - minute <= 5 

def answer_provider(row):
    if 'first' in row.match_period :
        return False 
    if 'stoppage' in row.match_period:
        return True 
    return is_end_of(row.match_period,row.minute_regulation)



In [54]:
goals['late_goal'] =  goals.apply(answer_provider,axis=1)

In [56]:
matches.to_csv('data/matches_combined_data.csv',index=False)
goals.to_csv('data/goals_data.csv',index=False)